In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.font_manager as fm
import math
import seaborn as sns
import os
from collections import Counter
import sklearn
from sklearn.decomposition import PCA
from sklearn import metrics, preprocessing, neighbors, cluster
from sklearn import cluster
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples, silhouette_score
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('ggplot')
# Create color maps
from matplotlib.colors import ListedColormap
cmap = ListedColormap(["#e41a1c","#984ea3","#a65628","#377eb8","#ffff33","#4daf4a","#ff7f00"])
%matplotlib inline
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv('Pull_Data.csv', encoding='utf-8')
df

In [ ]:
df2 = df.drop("rest name", axis=1)
rank_counts = df['rest name'].value_counts()
df2.hist(figsize=(50, 50))  # Set the figure size
plt.bar(rank_counts.index, rank_counts.values)  # Create the bar chart
plt.xlabel('Restaurant Rank')  # Set the x-axis label
plt.ylabel('Number of Restaurants')  # Set the y-axis label
plt.title('Distribution of Restaurant Ranks')  # Set the title
plt.show()  # Display the plot

In [ ]:
df['rest rank'].value_counts().plot(kind='pie')